In [44]:
!pip install mplcyberpunk

In [45]:
# ================================================================
# ANALISI CRIMINALITÀ & ECONOMIA ITALIA (2018–2023)
# ================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mplcyberpunk
import os

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

# directory grafici
os.makedirs("graphs", exist_ok=True)

print("✅ Librerie importate")

✅ Librerie importate


In [46]:
# ================================================================
# CONFIGURAZIONE
# ================================================================
BASE_URL = "https://raw.githubusercontent.com/dataxys/crime_economy_italy_analysis/main/data/raw"

print("📁 Base URL:", BASE_URL)

📁 Base URL: https://raw.githubusercontent.com/dataxys/crime_economy_italy_analysis/main/data/raw


In [47]:
# ================================================================
# FUNZIONE UNIVERSALE ISTAT EXCEL (2021–2023)
# ================================================================

def load_istat_delitti(url, anno):
    """
    Carica i file ISTAT INT00062 (2021–2023) con rilevazione automatica
    dello sheet e dell'header. Funziona su tutti gli Excel ISTAT.
    """
    try:
        # apre file excel
        xl = pd.ExcelFile(url)

        # trova sheet con 'prov' nel nome
        sheets = [s for s in xl.sheet_names if "prov" in s.lower()]
        sheet = sheets[0] if sheets else xl.sheet_names[0]

        df = pd.read_excel(xl, sheet_name=sheet)

        # identifica header con "provincia" nella riga
        header_idx = None
        for i in range(25):
            row = " ".join(df.iloc[i].astype(str).values).lower()
            if "prov" in row or "territ" in row:
                header_idx = i
                break

        if header_idx is None:
            print(f"❌ Header non trovato in {anno}")
            return None

        df.columns = df.iloc[header_idx]
        df = df.iloc[header_idx+1:]

        # trova colonne principali
        col_prov = next(c for c in df.columns if "prov" in str(c).lower())
        col_val = next(c for c in df.columns if "tot" in str(c).lower() or "num" in str(c).lower())
        col_reato = next(c for c in df.columns if "delitt" in str(c).lower() or "reat" in str(c).lower())

        df_clean = df[[col_prov, col_reato, col_val]].copy()
        df_clean.columns = ["Provincia", "reato", "totale_delitti"]

        df_clean["Anno"] = anno
        df_clean["regione"] = "N/A"

        df_clean["totale_delitti"] = pd.to_numeric(df_clean["totale_delitti"], errors="coerce")
        df_clean = df_clean.dropna(subset=["Provincia"])
        df_clean = df_clean[df_clean["totale_delitti"] > 0]

        print(f"  ✅ EXCEL {anno}: {df_clean.shape[0]} righe pulite")
        return df_clean

    except Exception as e:
        print(f"❌ Errore {anno}: {e}")
        return None

In [48]:
# ================================================================
# 1. POPOLAZIONE
# ================================================================

print("🔹 Caricamento popolazione POSAS...")

posas_url = f"{BASE_URL}/POSAS_2025_it_Province.csv"

posas = pd.read_csv(posas_url, sep=";", skiprows=1)
posas.columns = ["Codice", "Provincia", "Età", "Maschi", "Femmine", "Totale"]

for c in ["Maschi", "Femmine", "Totale"]:
    posas[c] = pd.to_numeric(posas[c], errors="coerce")

popolazione_prov = (
    posas.groupby("Provincia")["Totale"]
    .sum()
    .reset_index()
    .rename(columns={"Totale": "Popolazione"})
)

print(f"✅ Popolazione caricata: {len(popolazione_prov)} province")

🔹 Caricamento popolazione POSAS...
✅ Popolazione caricata: 107 province


In [49]:
# ================================================================
# 2. PIL
# ================================================================

print("🔹 Caricamento PIL...")

pil_url = f"{BASE_URL}/Tavole-allegate-2024_conti%20territoriali.xlsx"
pil_raw = pd.read_excel(pil_url, sheet_name="Tav. 8")

# trova header
header_row = pil_raw.index[pil_raw.apply(lambda r: r.astype(str).str.contains("territorio", case=False).any(), axis=1)][0]

pil = pil_raw.iloc[header_row+1:, :3]
pil.columns = ["Provincia", "PIL_2021", "PIL_2022"]

pil = pil[~pil["Provincia"].astype(str).str.contains("Totale|Italia|Nord|Sud|Centro", case=False)]
pil["Provincia"] = pil["Provincia"].astype(str).str.strip()

for c in ["PIL_2021", "PIL_2022"]:
    pil[c] = pd.to_numeric(pil[c], errors="coerce")

print(f"✅ PIL caricato: {len(pil)} province")

🔹 Caricamento PIL...
✅ PIL caricato: 106 province


In [50]:
# ================================================================
# 3. DELITTI - CSV 2018–2020
# ================================================================

delitti_list = []

print("🔹 Caricamento CSV delitti 2018–2020...\n")

for anno in [2018, 2019, 2020]:
    url = f"{BASE_URL}/delitti_{anno}.csv"
    df = pd.read_csv(url)
    df.columns = ["regione", "Provincia", "reato", "totale_delitti"]
    df["Anno"] = anno
    delitti_list.append(df)
    print(f"  ✅ CSV {anno}: {df.shape[0]} righe")

🔹 Caricamento CSV delitti 2018–2020...

  ✅ CSV 2018: 3604 righe
  ✅ CSV 2019: 3604 righe
  ✅ CSV 2020: 3604 righe


In [51]:
# ==========================
# CELLA 7 — VERSIONE DEFINITIVA E CORRETTA
# ==========================

import traceback

print("\n🔹 Caricamento Excel 2021–2023 (versione definitiva)...\n")

excel_files = {
    2021: f"{BASE_URL}/INT00062_Delitti_denunciati_2021_ITA-REG-PROV-CP.xlsx",
    2022: f"{BASE_URL}/INT00062_Delitti_denunciati_2022_ITA-REG-PROV-CP.xlsx",
    2023: f"{BASE_URL}/INT00062_Delitti_denunciati_2023_ITA-REG-PROV-CP.xlsx",
}

VALID_SHEETS = ["Province", "PROVINCE ELENCO"]

def clean_province_df(df_raw):
    """
    Dataframe ISTAT province – individua header corretto,
    standardizza nomi colonne e restituisce df pronto.
    """
    # trova riga header scorrendo le prime 40 righe
    header_idx = None
    for i in range(40):
        row = df_raw.iloc[i].astype(str).str.lower().values
        row_join = " ".join(row)
        if "prov" in row_join and ("den" in row_join or "delitt" in row_join):
            header_idx = i
            break

    if header_idx is None:
        return None

    header = df_raw.iloc[header_idx].values
    df = df_raw.iloc[header_idx+1:].copy()
    df.columns = header
    df = df.rename(columns=lambda c: str(c).strip())

    # individua colonne chiave
    col_prov = next((c for c in df.columns if "prov" in c.lower()), None)
    col_val = next((c for c in df.columns if "tot" in c.lower() or "den" in c.lower()), None)
    col_reato = next((c for c in df.columns if "delitt" in c.lower() or "reat" in c.lower()), None)

    if not col_prov or not col_val:
        return None

    df_clean = df[[col_prov, col_val] + ([col_reato] if col_reato else [])].copy()
    df_clean.columns = ["Provincia", "totale_delitti"] + (["reato"] if col_reato else [])

    # pulizia
    df_clean["Provincia"] = df_clean["Provincia"].astype(str).str.strip().str.title()
    df_clean["totale_delitti"] = pd.to_numeric(df_clean["totale_delitti"], errors="coerce").fillna(0)
    df_clean = df_clean[df_clean["Provincia"].str.strip().astype(bool)]

    if "reato" not in df_clean.columns:
        df_clean["reato"] = "Totale"

    return df_clean


for anno, url in excel_files.items():
    print(f"📘 Elaborazione Excel {anno}")

    try:
        xl = pd.ExcelFile(url)
        sheets = [s for s in xl.sheet_names if s in VALID_SHEETS]

        print(f"  Sheets filtrate: {sheets}")

        df_success = None

        for s in sheets:
            raw = pd.read_excel(xl, sheet_name=s, header=None)
            parsed = clean_province_df(raw)

            if parsed is not None:
                df_success = parsed.copy()
                print(f"  ✅ Sheet valida trovata: {s} ({len(parsed)} righe)")
                break
            else:
                print(f"  ⚠️ Sheet {s} scartata: header non riconosciuto")

        if df_success is None:
            print(f"  ❌ Nessuna delle sheet valide contiene dati riconoscibili per {anno}")
            continue

        df_success["Anno"] = anno
        delitti_list.append(df_success[["Provincia", "reato", "totale_delitti", "Anno"]])

    except Exception as e:
        print(f"  ❌ Errore {anno}: {e}")
        print(traceback.format_exc())


🔹 Caricamento Excel 2021–2023 (versione definitiva)...

📘 Elaborazione Excel 2021


/usr/local/lib/python3.12/dist-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'CAPOLUOGHI PROV ELENCO'!$9:$8276.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")


  Sheets filtrate: ['Province', 'PROVINCE ELENCO']
  ✅ Sheet valida trovata: Province (108 righe)
📘 Elaborazione Excel 2022


/usr/local/lib/python3.12/dist-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'CAPOLUOGHI PROV ELENCO'!$9:$8276.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")


  Sheets filtrate: ['Province', 'PROVINCE ELENCO']
  ✅ Sheet valida trovata: Province (108 righe)
📘 Elaborazione Excel 2023
  Sheets filtrate: ['Province', 'PROVINCE ELENCO']
  ✅ Sheet valida trovata: Province (108 righe)


/usr/local/lib/python3.12/dist-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'CAPOLUOGHI PROV ELENCO'!$9:$8276.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")


In [52]:
# ================================================================
# 4. UNIFICAZIONE & SALVATAGGIO
# ================================================================

print("\n🔗 Unificazione dataset delitti...")

delitti_raw = pd.concat(delitti_list, ignore_index=True)

# pulizia base
delitti_raw["totale_delitti"] = pd.to_numeric(delitti_raw["totale_delitti"], errors="coerce").fillna(0)
delitti_raw["Provincia"] = delitti_raw["Provincia"].astype(str).str.title().str.strip()

delitti_aggregati = (
    delitti_raw.groupby(["Provincia", "Anno"])["totale_delitti"]
    .sum()
    .reset_index()
)

delitti_aggregati.to_csv("delitti_completi_2018_2023.csv", index=False)

print(f"💾 Salvato: delitti_completi_2018_2023.csv")


🔗 Unificazione dataset delitti...
💾 Salvato: delitti_completi_2018_2023.csv


In [53]:
# ================================================================
# 5. MERGE & NORMALIZZAZIONE
# ================================================================

print("\n🔗 Merge dati finali...")

df = (
    delitti_aggregati
    .merge(popolazione_prov, on="Provincia", how="left")
    .merge(pil[["Provincia", "PIL_2022"]], on="Provincia", how="left")
)

df["crimini_per_100k"] = df["totale_delitti"] / df["Popolazione"] * 100_000

df.to_csv("dataset_finale_powerbi.csv", index=False)

print(f"💾 Dataset finale salvato: dataset_finale_powerbi.csv")
print(df.head())


🔗 Merge dati finali...
💾 Dataset finale salvato: dataset_finale_powerbi.csv
   Provincia  Anno  totale_delitti  Popolazione   PIL_2022  crimini_per_100k
0  Agrigento  2018          2543.0     816118.0  17.961399        311.597097
1  Agrigento  2019          2314.0     816118.0  17.961399        283.537430
2  Agrigento  2020          1991.0     816118.0  17.961399        243.959820
3  Agrigento  2021         10483.0     816118.0  17.961399       1284.495624
4  Agrigento  2022         10732.0     816118.0  17.961399       1315.005918


In [56]:
# ==========================
# CELLA 10 — GRAFICI (2018–2023)
# ==========================
import matplotlib.pyplot as plt
import seaborn as sns
import mplcyberpunk
import os

plt.style.use("cyberpunk")
os.makedirs("graphs", exist_ok=True)

print("\n🎨 GENERAZIONE GRAFICI CYBERPUNK...\n")

# =======================================================
# 1) TREND NAZIONALE 2018–2023
# =======================================================
trend = df.groupby("Anno").agg({
    "totale_delitti": "sum",
    "Popolazione": "sum"
}).reset_index()

trend["crimini_per_100k"] = trend["totale_delitti"] / trend["Popolazione"] * 100000

plt.figure(figsize=(10, 5))
plt.plot(trend["Anno"], trend["crimini_per_100k"], marker="o")
plt.title("Andamento nazionale dei crimini (2018–2023)")
plt.ylabel("Crimini per 100.000 abitanti")
mplcyberpunk.add_glow_effects()
plt.tight_layout()
plt.savefig("graphs/01_trend_nazionale.png")
plt.close()

print("✅ Salvato: graphs/01_trend_nazionale.png")


# =======================================================
# 2) TOP 15 PROVINCE (media 2018–2023)
# =======================================================
top15 = df.groupby("Provincia")["crimini_per_100k"].mean().nlargest(15).sort_values()

plt.figure(figsize=(10, 6))
plt.barh(top15.index, top15.values)
plt.title("Top 15 province per crimini per 100.000 abitanti (media 2018–2023)")
mplcyberpunk.add_glow_effects()
plt.tight_layout()
plt.savefig("graphs/02_top15_province.png")
plt.close()

print("✅ Salvato: graphs/02_top15_province.png")


# =======================================================
# 3) SCATTER PLOT — Crimini vs PIL
# =======================================================
plt.figure(figsize=(8, 6))
for anno in sorted(df["Anno"].unique()):
    tmp = df[df["Anno"] == anno]
    plt.scatter(tmp["PIL_2022"], tmp["crimini_per_100k"], alpha=0.8, label=str(anno))

plt.title("Relazione tra PIL provinciale e criminalità (2018–2023)")
plt.xlabel("PIL pro capite (2022)")
plt.ylabel("Crimini per 100.000 abitanti")
plt.legend(title="Anno")
mplcyberpunk.add_glow_effects()
plt.tight_layout()
plt.savefig("graphs/03_crimini_vs_pil.png")
plt.close()

print("✅ Salvato: graphs/03_crimini_vs_pil.png")


# =======================================================
# 4) HEATMAP — Evoluzione criminalità
# =======================================================
heatmap_df = df.pivot_table(
    index="Provincia",
    columns="Anno",
    values="crimini_per_100k",
    aggfunc="mean"
)

plt.figure(figsize=(14, 22))
sns.heatmap(heatmap_df, cmap="magma", linewidths=0.1)
plt.title("Evoluzione della criminalità per provincia (2018–2023)")
plt.tight_layout()
plt.savefig("graphs/04_heatmap_evoluzione.png")
plt.close()

print("✅ Salvato: graphs/04_heatmap_evoluzione.png")

print("\n🎉 GRAFICI CYBERPUNK GENERATI!")


🎨 GENERAZIONE GRAFICI CYBERPUNK...

✅ Salvato: graphs/01_trend_nazionale.png
✅ Salvato: graphs/02_top15_province.png
✅ Salvato: graphs/03_crimini_vs_pil.png
✅ Salvato: graphs/04_heatmap_evoluzione.png

🎉 GRAFICI CYBERPUNK GENERATI!


In [ ]:
X = df_finale[['pil_pro_capite', 'densita_abitativa', 'spesa_pubblica']]
X = sm.add_constant(X)
y = df_finale['delitti_100k']

model = sm.OLS(y, X).fit()
print(model.summary())